In [1]:
!if [ ! -d mov-in-covid ]; then \
  git clone https://github.com/tanaxer01/mov-in-covid; \
  mv mov-in-covid/*.py .; fi

from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import numpy  as np
import utils

Cloning into 'mov-in-covid'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 13 (delta 3), reused 11 (delta 1), pack-reused 0
Unpacking objects: 100% (13/13), done.


In [2]:
datasets = {
  # DP51
  "dp51_difs": "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto51/ISCI_std.csv",
  # DP82
  "dp82_weeks": "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto82/ISCI_weeks.csv",
  "dp82_weekends": "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto82/ISCI_weekends.csv",
  # DP...
}

for name, url in datasets.items():
  utils.get_file(name, url)

[+] created ./datos/
[+] downloaded ./datos/dp51_difs
[+] downloaded ./datos/dp82_weeks
[+] downloaded ./datos/dp82_weekends


# DATASET PREP

## DP51

In [3]:
df_difs = pd.read_csv("./datos/dp51_difs.csv")

# Just RM
df_difs = df_difs[ df_difs['Codigo region'] == 13 ].drop(columns=['Region', 'Codigo region'])

df_difs['Fecha'] = pd.to_datetime(df_difs['Fecha'])
df_difs = df_difs.sort_values(by=['Fecha'])

def parse_diff(x):
  a, b = [ int(i) for i in x[1:-2].replace('%','').split(",") ]
  return (b+a)/2

df_difs['salida'] = df_difs['Dif salida'].map(parse_diff)

salida_comuna = df_difs[ df_difs['Comuna'] == 'Santiago' ].groupby('Fecha')['salida'].mean()

salida_comuna

Fecha
2020-03-02    10.174419
2020-03-09    10.174419
2020-03-16     2.732558
2020-03-23   -15.096899
2020-03-30   -20.910853
                ...    
2022-03-28     9.717054
2022-04-04     8.321705
2022-04-11     9.717054
2022-04-18     8.321705
2022-04-25    10.422481
Name: salida, Length: 110, dtype: float64

## DP82

In [23]:
df_weeks = pd.read_csv('./datos/dp82_weeks.csv')
#df_weekends = pd.read_csv('./datos/dp82_weekends.csv')

# Just RM
df_weeks = df_weeks[ df_weeks['region'] == 13 ].drop(columns=['region'])

#df_weeks['fecha_inicio'] = pd.to_datetime(df_weeks['fecha_inicio'])
#df_weeks = df_weeks.sort_values(by=['fecha_inicio'])

#df_weeks[ df_weeks['nom_comuna'] == 'SANTIAGO' ].head()

In [24]:
df_weeks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6120 entries, 8784 to 14903
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   semana                     6120 non-null   int64  
 1   paso                       5304 non-null   float64
 2   nom_comuna                 6120 non-null   object 
 3   comuna                     6120 non-null   int64  
 4   fecha_inicio               6120 non-null   object 
 5   fecha_termino              6120 non-null   object 
 6   var_salidas                6120 non-null   float64
 7   var_salidas_cota_inferior  6120 non-null   float64
 8   var_salidas_cota_superior  6120 non-null   float64
dtypes: float64(4), int64(2), object(3)
memory usage: 478.1+ KB


In [32]:
print(df_weeks[ ~np.isnan(df_weeks['paso']) ]['semana'].iloc[[0, -1]])
print(df_weeks[ np.isnan(df_weeks['paso']) ]['semana'].iloc[[0, -1]])

8784       9
14888    117
Name: semana, dtype: int64
8876     105
14903    132
Name: semana, dtype: int64
